In [30]:
import numpy as np
import random
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from qiskit.quantum_info import DensityMatrix, random_density_matrix
from qiskit.quantum_info.operators import Operator
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.linalg import sqrtm

# First Attempt, Bare Bones Neural Net

# Import the Dataset

In [2]:

QST_data = pd.read_csv("../data/qst_dataset.csv")
N = QST_data.shape[0]
QST_data.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31
0,0.173828,0.274414,0.490234,0.061523,0.175781,0.259766,0.495117,0.069336,0.318359,0.133789,...,-0.011859,0.074632,0.014527,0.011859,-1.032135e-17,-0.156679,-0.083966,-0.074632,0.156679,2.956436e-18
1,0.269531,0.179688,0.277344,0.273438,0.293945,0.157227,0.257812,0.291016,0.179688,0.248047,...,-0.031100,0.056268,-0.136595,0.031100,-1.532738e-18,-0.145086,-0.101096,-0.056268,0.145086,1.936221e-18
2,0.204102,0.483398,0.210938,0.101562,0.256836,0.434570,0.201172,0.107422,0.196289,0.448242,...,-0.042311,-0.027411,0.095658,0.042311,2.765645e-18,-0.054666,0.046359,0.027411,0.054666,-2.587528e-18
3,0.389648,0.251953,0.078125,0.280273,0.411133,0.244141,0.084961,0.259766,0.488281,0.160156,...,0.104071,0.019399,0.073657,-0.104071,3.922884e-19,0.084765,-0.038264,-0.019399,-0.084765,-5.285627e-18
4,0.382812,0.276367,0.140625,0.200195,0.377930,0.229492,0.182617,0.209961,0.426758,0.207031,...,-0.165184,-0.025058,-0.023143,0.165184,5.761045e-18,-0.082392,0.058770,0.025058,0.082392,-3.947200e-18


In [3]:
# clip any small values to zero
eps = 1e-10

#create boolean mask
mask = QST_data.abs() < eps
QST_data[mask] =0

In [4]:
X_cols = []
y_cols = []

N_x = 36
N_y = 32
for i in range(N_x):
    X_cols.append(f"x{i}")
for i in range(N_y):
    y_cols.append(f"y{i}")

X = QST_data[X_cols]
y = QST_data[y_cols]

#split the data
X_train, X_test, y_train, y_test=  train_test_split(X, y, train_size=0.7, random_state=42)


In [5]:
X_train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35
836,0.538086,0.216797,0.167969,0.077148,0.497070,0.247070,0.185547,0.070312,0.236328,0.476562,...,0.206055,0.236328,0.496094,0.090820,0.200195,0.212891,0.218750,0.350586,0.168945,0.261719
575,0.255859,0.140625,0.184570,0.418945,0.270508,0.142578,0.180664,0.406250,0.227539,0.177734,...,0.383789,0.339844,0.087891,0.190430,0.362305,0.359375,0.208008,0.091797,0.301758,0.398438
557,0.274414,0.290039,0.215820,0.219727,0.298828,0.320312,0.200195,0.180664,0.087891,0.506836,...,0.362305,0.442383,0.111328,0.056641,0.388672,0.443359,0.040039,0.141602,0.073242,0.745117
1235,0.328125,0.177734,0.158203,0.335938,0.301758,0.163086,0.184570,0.350586,0.317383,0.153320,...,0.285156,0.161133,0.273438,0.278320,0.266602,0.181641,0.403320,0.164062,0.368164,0.064453
1360,0.278320,0.300781,0.204102,0.216797,0.314453,0.280273,0.171875,0.233398,0.133789,0.475586,...,0.268555,0.177734,0.199219,0.348633,0.272461,0.179688,0.231445,0.345703,0.174805,0.248047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,0.290039,0.147461,0.307617,0.254883,0.306641,0.153320,0.307617,0.232422,0.333008,0.146484,...,0.344727,0.094727,0.274414,0.255859,0.348633,0.121094,0.169922,0.392578,0.308594,0.128906
1294,0.151367,0.365234,0.085938,0.397461,0.113281,0.409180,0.090820,0.386719,0.339844,0.180664,...,0.058594,0.197266,0.143555,0.612305,0.062500,0.181641,0.483398,0.251953,0.112305,0.152344
860,0.206055,0.248047,0.131836,0.414062,0.206055,0.212891,0.144531,0.436523,0.286133,0.137695,...,0.191406,0.479492,0.127930,0.202148,0.215820,0.454102,0.142578,0.186523,0.325195,0.345703
1459,0.221680,0.118164,0.233398,0.426758,0.214844,0.112305,0.228516,0.444336,0.268555,0.047852,...,0.344727,0.215820,0.091797,0.320312,0.364258,0.223633,0.258789,0.169922,0.293945,0.277344


# Training the NN

In [6]:
QST_NN = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(N_x,)),
    tf.keras.layers.Dense(50, activation="relu"),       # 2 dense layers, TOBE modified
    tf.keras.layers.Dense(50, activation="relu"),       # 2 dense layers, TOBE modified
    tf.keras.layers.Dense(N_y)
])


In [7]:
QST_NN.compile(
    optimizer = 'adam',
    loss = 'mse',
    metrics=['mse']
)

In [8]:
QST_NN.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1
)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0160 - mse: 0.0160 - val_loss: 0.0070 - val_mse: 0.0070
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.0060 - val_mse: 0.0060
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0053 - val_mse: 0.0053
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0046 - val_mse: 0.0046
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.0042 - val_mse: 0.0042
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0039 - val_mse: 0.0039
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.0037 - val_mse: 0.0037
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0035 - val_mse: 0.0035
Epoch 9/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - loss: 

In [9]:
y_pred = QST_NN.predict(X_test)

test_loss, _ = QST_NN.evaluate(X_test, y_test)

print(f"test loss: {test_loss}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 327us/step - loss: 0.0033 - mse: 0.0033
test loss: 0.0033153421245515347


## Evaluation metrics

In [10]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2: : {r2}")


Mean Squared Error: 0.003315341986300192
R^2: : 0.4169708490371704


# Second Attempt: Using more Advanced Methods

## 1) Data PreProccessing
    
- Cholesky Decomposition on targets rho
- Principal component analysis on data X (frequencies)

In [ ]:
X = X.to_numpy()
y = y.to_numpy()

## 2) Building Advanced Neural Network 
- Batch Normalisation
- Enforce positivity and Trace 1 on outputs via cholesky decomposition on the inputs
- Regularisation

In [ ]:


def vec_to_rho(y_vec):
    real = y_vec[:16].reshape(4, 4)
    imag = y_vec[16:].reshape(4, 4)
    return real + 1j*imag

def rho_to_alpha(rho):
    # T is lower triangular matrix
    L = np.linalg.cholesky(rho)
    alpha = []
    # extract the reals on the diagonals
    for i in range(rho.shape[0]):
        alpha.append(np.real(L[i, i]))  # add them to alpha
    
    # Off diagonals, contain real and imag components
    for i in range(1, rho.shape[0]):
        for j in range(i):
            alpha.append(np.real(L[i, j]))
            alpha.append(np.imag(L[i, j]))
    return np.array(alpha)

# build the full target array
alphas = np.stack([ rho_to_alpha(vec_to_rho(y[i])) for i in range(len(y)) ], axis=0)

N_x = X.shape[1]   # 36
N_alpha = alphas.shape[1]  # 16

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(N_x,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(N_alpha)   
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="mse",
    metrics=["mse"]
)

history = model.fit(
    X, alphas,
    validation_split=0.2,
    epochs=1000,
    batch_size=1000,
)



Epoch 1/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - loss: 1.2880 - mse: 1.2880 - val_loss: 0.0812 - val_mse: 0.0812
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.9430 - mse: 0.9430 - val_loss: 0.0754 - val_mse: 0.0754
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.7129 - mse: 0.7129 - val_loss: 0.0712 - val_mse: 0.0712
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.5608 - mse: 0.5608 - val_loss: 0.0680 - val_mse: 0.0680
Epoch 5/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.4767 - mse: 0.4767 - val_loss: 0.0656 - val_mse: 0.0656
Epoch 6/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.3983 - mse: 0.3983 - val_loss: 0.0636 - val_mse: 0.0636
Epoch 7/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.3508 - mse: 0.3508 - val_loss: 0.0618 - val_mse: 0.0618
Epoch 8/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.3208 - mse: 0.3208 - val_loss: 0.0603 - val_mse: 0.0603
Epoch 9/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.30

In [ ]:
def alpha_to_rho(alpha):
    """Convert batch of alpha vectors to density matrices using Cholesky."""
    N = alpha.shape[0]
    rho = np.zeros((N, 4, 4), dtype=np.complex64)
    for i in range(N):
        a = alpha[i]
        L = np.zeros((4, 4), dtype=np.complex64)
        idx = 0
        for j in range(4):
            L[j, j] = a[idx]
            idx += 1
        for j in range(1, 4):
            for k in range(j):
                re = a[idx]
                im = a[idx + 1]
                L[j, k] = re + 1j * im
                idx += 2
        rho_i = L @ L.conj().T
        rho[i] = rho_i / np.trace(rho_i)
    return rho

def reconstruct_true_rho(y):
    """Convert vectorized real+imag back to complex 4x4 matrices."""
    N = y.shape[0]
    real = y[:, :16].reshape(N, 4, 4)
    imag = y[:, 16:].reshape(N, 4, 4)
    return real + 1j * imag

def fidelity(rho1, rho2):
    """
    Uhlmann fidelity between two density matrices.
    inputs: the two density matrices to be compares
    """
    sqrt_rho1 = sqrtm(rho1)
    product = sqrt_rho1 @ rho2 @ sqrt_rho1
    sqrt_product = sqrtm(product)
    F = np.trace(sqrt_product)
    return np.real(F)**2


alpha_pred = model.predict(X_test)      
rho_pred = alpha_to_rho(alpha_pred)  
rho_true = reconstruct_true_rho(y_test) 

# Compute fidelities
fidelities = np.array([
    fidelity(rho_pred[i], rho_true[i])
    for i in range(len(rho_pred))
])

# Summary
mean_fid = np.mean(fidelities)
std_fid = np.std(fidelities)
print(f"Test Set Fidelity: {mean_fid:.4f} ± {std_fid:.4f}")


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Test Set Fidelity: 0.8657 ± 0.0522


In [14]:
y_test

,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,...,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31
1860,0.229030,-0.115303,0.025835,-0.065672,-0.115303,0.452378,0.069032,-0.144452,0.025835,0.069032,...,0.018472,-0.024981,0.046703,-0.018472,0.0,0.033897,0.042982,0.024981,-0.033897,0.0
353,0.168633,-0.082622,0.126550,-0.016326,-0.082622,0.410598,-0.072670,0.195494,0.126550,-0.072670,...,0.096532,-0.008276,0.076030,-0.096532,0.0,0.036845,-0.141605,0.008276,-0.036845,0.0
1333,0.143883,0.119693,0.009683,-0.024171,0.119693,0.211563,-0.042449,0.112700,0.009683,-0.042449,...,-0.093015,0.061152,0.143081,0.093015,0.0,-0.185343,-0.024787,-0.061152,0.185343,0.0
905,0.230750,0.087305,0.063252,-0.105117,0.087305,0.280255,-0.005413,-0.009406,0.063252,-0.005413,...,-0.099613,-0.075495,0.070375,0.099613,0.0,-0.056051,0.098093,0.075495,0.056051,0.0
1289,0.170142,0.054815,0.103683,-0.064274,0.054815,0.414075,-0.054411,-0.108939,0.103683,-0.054411,...,-0.061165,0.027946,0.113398,0.061165,0.0,-0.081950,0.014961,-0.027946,0.081950,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,0.183423,-0.168161,0.052786,-0.000764,-0.168161,0.345395,-0.000085,-0.013357,0.052786,-0.000085,...,-0.018560,-0.123151,-0.041173,0.018560,0.0,-0.162566,0.061188,0.123151,0.162566,0.0
1554,0.332239,0.090484,-0.144702,-0.003779,0.090484,0.229147,-0.197319,0.017807,-0.144702,-0.197319,...,0.036561,0.062303,0.026394,-0.036561,0.0,-0.098389,-0.066791,-0.062303,0.098389,0.0
1956,0.268646,-0.153508,-0.011184,-0.033492,-0.153508,0.208840,0.046951,-0.054793,-0.011184,0.046951,...,-0.004181,-0.052547,0.016078,0.004181,0.0,-0.072885,-0.145092,0.052547,0.072885,0.0
925,0.149837,0.049939,0.126156,-0.047087,0.049939,0.388340,0.015009,-0.078361,0.126156,0.015009,...,-0.146485,-0.081759,-0.011955,0.146485,0.0,0.080416,-0.044935,0.081759,-0.080416,0.0
